In [1]:
#using Pkg
#Pkg.add("Profile")

In [2]:
using DelimitedFiles
using LinearAlgebra
using Optim
using Distributions
using Test
using BenchmarkTools
using Random
using CSV
using DataFrames
using Distributed
using Plots
using Profile

In [3]:
pwd()

"/Users/FredYu/Documents/GitHub/BulkLMM.jl/analysis"

In [94]:
## Include the source code of BulkLMM to be tested:
include("../src/scan.jl");
include("../src/lmm.jl");
include("../src/wls.jl");
include("../src/util.jl");
include("../src/kinship.jl");
include("../src/readData.jl");

## Also include the helper functions for writing tests:
include("../test/testHelper.jl");

In [5]:
## Read in BXD data:
pheno_file = "../data/bxdData/BXDtraits.csv"
pheno = readBXDpheno(pheno_file);
geno_file = "../data/bxdData/BXDgeno_prob.csv"
geno = readGenoProb_ExcludeComplements(geno_file);

kinship = calcKinship(geno); # calculate kinship matrix from genotype data

In [6]:
nperms = 1024; # number of permutated copies required;
m = size(pheno, 2);
p = size(geno, 2); # number of markers

In [7]:
## Consider the 7919-th trait
pheno_y = reshape(pheno[:, 7919], :, 1);

## Do the same for many traits (first 1000):

In [8]:
# Function that extracts the row maximums for a matrix:
function getMaxLod_manyTraits(pheno::Array{Float64, 2}, geno::Array{Float64, 2}, kinship::Array{Float64, 2};
                              ntraits::Int64 = 1000, nperms::Int64 = 1024, original::Bool = true)
    
    if original
        nrows = nperms+1
    else
        nrows = nperms
    end
    
    maxLods_manyTraits = Matrix{Float64}(undef, nrows, ntraits);
    
    for k in 1:ntraits
        
        pheno_k = reshape(pheno[:, k], :, 1);
        results = scan_perms(pheno_k, geno, kinship; nperms = nperms)
        
        maxLods_manyTraits[:, k] = getMaxLod(results)
    end
    
    return maxLods_manyTraits; 
end


function getMaxLod(results::Array{Float64, 2})
    
    return mapslices(x -> maximum(x), results; dims = 2);
    
end

getMaxLod (generic function with 1 method)

In [85]:
include("../src/scan.jl");

In [10]:
@benchmark scan_perms(pheno_y, geno, kinship; nperms = 1024)

BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 7.791 s (10.34% GC) to evaluate,
 with a memory estimate of 13.63 GiB, over 194772 allocations.

In [26]:
@benchmark scan_perms(pheno_y, geno, kinship; nperms = 1024)

BenchmarkTools.Trial: 1 sample with 1 evaluation.
 Single result which took 8.179 s (11.35% GC) to evaluate,
 with a memory estimate of 13.63 GiB, over 194772 allocations.

In [87]:
include("../src/scan_for_tests.jl");

In [51]:
(y_st, X_st) = permuteHelper(pheno_y, geno, kinship);

In [53]:
calcLod_perms(y_st, X_st[:, 1])[1:6]

6-element Vector{Float64}:
 0.008196360329331354
 1.7191295863542346
 0.22597355989446322
 0.016780778180664147
 0.08000667915488369
 0.00035453485562264575

In [54]:
calcLod_perms2(y_st, X_st[:, 1])[1:6]

6-element Vector{Float64}:
 0.008196360329331354
 1.7191295863542346
 0.22597355989446322
 0.016780778180664147
 0.08000667915489246
 0.00035453485562264575

In [55]:
calcLod_perms3(y_st, X_st[:, 1])[1:6]

6-element Vector{Float64}:
 0.008196360329331354
 1.7191295863542346
 0.22597355989446322
 0.016780778180664147
 0.08000667915489246
 0.00035453485562264575

In [56]:
@benchmark calcLod_perms(y_st, X_st[:, 1])

BenchmarkTools.Trial: 2908 samples with 1 evaluation.
 Range (min … max):  783.781 μs …   7.694 ms  ┊ GC (min … max):  0.00% … 67.10%
 Time  (median):       1.579 ms               ┊ GC (median):     0.00%
 Time  (mean ± σ):     1.704 ms ± 923.700 μs  ┊ GC (mean ± σ):  10.51% ± 15.04%

          █▃                                                     
  ▇▄▃▃▃▃▂▄██▇▆▅▄▄▃▂▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▂▂▂▂▂▂▂▂▂▂▂ ▃
  784 μs           Histogram: frequency by time         6.13 ms <

 Memory estimate: 2.87 MiB, allocs estimate: 10827.

In [57]:
@benchmark calcLod_perms2(y_st, X_st[:, 1])

BenchmarkTools.Trial: 4125 samples with 1 evaluation.
 Range (min … max):  257.348 μs …   7.951 ms  ┊ GC (min … max):  0.00% … 83.22%
 Time  (median):       1.064 ms               ┊ GC (median):     0.00%
 Time  (mean ± σ):     1.196 ms ± 941.721 μs  ┊ GC (mean ± σ):  12.11% ± 13.57%

        ▄█▂                                                      
  ▇▃▃▂▂▂████▆▅▄▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▂▁▂▂▂▂▂ ▃
  257 μs           Histogram: frequency by time         6.68 ms <

 Memory estimate: 1.91 MiB, allocs estimate: 22.

In [58]:
@benchmark calcLod_perms3(y_st, X_st[:, 1])

BenchmarkTools.Trial: 4556 samples with 1 evaluation.
 Range (min … max):  244.127 μs …   7.198 ms  ┊ GC (min … max):  0.00% … 85.80%
 Time  (median):     983.613 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):     1.087 ms ± 850.113 μs  ┊ GC (mean ± σ):  12.05% ± 13.46%

  ▅▂    ▃█▆▅▅▃▃▁                                                ▁
  ███▅▅▁████████▇▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▆▆▆▇▆▇▇ █
  244 μs        Histogram: log(frequency) by time        670 ms <

 Memory estimate: 1.90 MiB, allocs estimate: 20.

In [81]:
@btime calcLod_perms(y_st, X_st[:, 1]);

  781.570 μs (10827 allocations: 2.87 MiB)


In [82]:
@btime calcLod_perms2(y_st, X_st[:, 1]);

  261.859 μs (22 allocations: 1.91 MiB)


In [83]:
@btime calcLod_perms3(y_st, X_st[:, 1]);

  248.826 μs (20 allocations: 1.90 MiB)


In [91]:
@btime scan_perms_toCompare(pheno_y, geno, kinship; choice = 1);

  14.360 s (79258602 allocations: 20.52 GiB)


In [92]:
@btime scan_perms_toCompare(pheno_y, geno, kinship; choice = 2);

  9.922 s (155195 allocations: 13.69 GiB)


In [93]:
@btime scan_perms_toCompare(pheno_y, geno, kinship; choice = 3);

  8.821 s (140553 allocations: 13.64 GiB)


In [126]:
include("../src/wls.jl");

In [127]:
include("../src/scan.jl");

In [133]:
include("../src/scan_for_tests.jl");

In [130]:
@btime scan_perms(pheno_y, geno, kinship) # oldest

  9.061 s (215712 allocations: 13.63 GiB)


1025×7321 Matrix{Float64}:
 0.00819636   0.00819636   0.00819636   …  0.0128283    0.0128283
 1.71913      1.71913      1.71913         0.452205     0.452205
 0.225974     0.225974     0.225974        0.201861     0.201861
 0.0167808    0.0167808    0.0167808       0.00762492   0.00762492
 0.0800067    0.0800067    0.0800067       0.00972036   0.00972036
 0.000354535  0.000354535  0.000354535  …  0.383925     0.383925
 0.58573      0.58573      0.58573         0.202066     0.202066
 0.519134     0.519134     0.519134        0.0839292    0.0839292
 3.18899e-5   3.18899e-5   3.18899e-5      0.144341     0.144341
 1.03257      1.03257      1.03257         0.183653     0.183653
 0.0559665    0.0559665    0.0559665    …  0.0507086    0.0507086
 0.0760841    0.0760841    0.0760841       0.0705379    0.0705379
 0.0890595    0.0890595    0.0890595       0.167509     0.167509
 ⋮                                      ⋱               ⋮
 0.156918     0.156918     0.156918        0.212798     0.2127

In [124]:
@btime scan_perms2(pheno_y, geno, kinship); # new

  7.908 s (193749 allocations: 13.62 GiB)


In [129]:
@btime scan_perms2(pheno_y, geno, kinship);

  7.949 s (193749 allocations: 13.62 GiB)


In [ ]:
@btime scan_perms2(pheno_y, geno, kinship); # newest

In [ ]:
scan_perms2(pheno_y, geno, kinship)